In [1]:
import cv2
import os
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def train_svm(hog_descriptors, labels):
    """Entrena un clasificador SVM con los descriptores HOG."""
    # Escalar los descriptores
    scaler = StandardScaler()
    hog_descriptors_scaled = scaler.fit_transform(hog_descriptors)

    # Dividir datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(hog_descriptors_scaled, labels, test_size=0.2, random_state=42)

    # Entrenar el clasificador SVM
    clf = svm.SVC(kernel='linear', C=1.0)
    clf.fit(X_train, y_train)

    # Evaluar el modelo
    accuracy = clf.score(X_test, y_test)
    print(f"Precisión del modelo SVM: {accuracy:.2f}")

    return clf, scaler  # Devolver también el scaler


def load_images_and_labels(path, hog):
    """Carga las imágenes y calcula los descriptores HOG."""
    images = []
    classNames = []
    hog_descriptors = []

    for cl in os.listdir(path):
        img = cv2.imread(os.path.join(path, cl))
        if img is None:
            print(f"No se pudo cargar la imagen '{cl}'.")
            continue
        img = cv2.resize(img, hog.winSize)

        # Preprocesamiento de imágenes
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_gray = cv2.equalizeHist(img_gray)  # Equalización de histograma
        img_processed = cv2.GaussianBlur(img_gray, (5, 5), 0)  # Suavizado Gaussiano
        images.append(img_processed)
        classNames.append(os.path.splitext(cl)[0])

        # Calcular descriptores HOG
        descriptor = hog.compute(img_processed)
        hog_descriptors.append(descriptor.flatten())

    return images, classNames, hog_descriptors

def create_hog_descriptor(winSize=(64, 64), blockSize=(16, 16), blockStride=(8, 8),
                          cellSize=(8, 8), nbins=9, derivAperture=1, winSigma=-1.,
                          histogramNormType=0, L2HysThreshold=0.2, gammaCorrection=1,
                          nlevels=64):
    """Crea y devuelve un objeto HOGDescriptor con los parámetros especificados."""
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins, derivAperture,
                            winSigma, histogramNormType, L2HysThreshold, gammaCorrection, nlevels)
    return hog

def main():
    path = "../ORB_Img/ImagesQuery/"

    # Crear el objeto HOGDescriptor y cargar imágenes
    hog = create_hog_descriptor(winSize=(128, 128))
    images, classNames, hog_descriptors = load_images_and_labels(path, hog)

    # Crear etiquetas
    labels = list(range(len(images)))

   # Entrenar el clasificador SVM
    clf, scaler = train_svm(hog_descriptors, labels)  # Capturar también el scaler

    # Iniciar la captura de video desde la cámara
    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("No se pudo abrir la cámara.")
        exit()

    while True:
        success, img = cap.read()
        if not success:
            break

        img_original = img.copy()
        img = cv2.resize(img, hog.winSize)

        # Preprocesamiento de imágenes para detección en tiempo real
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img_gray = cv2.equalizeHist(img_gray)
        img_processed = cv2.GaussianBlur(img_gray, (5, 5), 0)

        # Calcular descriptores HOG y predecir etiqueta
        descriptor = hog.compute(img_processed)
        descriptor_flatten = descriptor.flatten().reshape(1, -1)
        descriptor_scaled = scaler.transform(descriptor_flatten)  # Escalar descriptor

        label = clf.predict(descriptor_scaled)
        name = classNames[label[0]]

        cv2.putText(img_original, str(name), (10, 50), cv2.FONT_HERSHEY_COMPLEX, 1,(0,0,255),1)#, cv2.LINE_AA)
        cv2.imshow('Detector HOG', img_original)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Precisión del modelo SVM: 0.00


# Este codigo no es originalmente ideado por mi, sugiero que se utilice dando creditos a otros.